# Unscented transform






In [156]:
import numpy as np
import matplotlib.pyplot as plt

In [157]:
def van_der_merwe_weights(size, alpha, kappa, beta):
  lmbd = alpha ** 2 * (size + kappa) - kappa
  wi = 1 / (2 * (size + lmbd))
  wm = np.full(2 * size + 1, wi)
  wc = np.full(2 * size + 1, wi)
  wm[0] = lmbd / (size + lmbd)
  wc[0] = wm[0] + (1 - alpha **2 - beta)
  return wm, wc

def unscented_transform(transform_functor, mean, cov, mean_weights, cov_weights, alpha, kappa):
  # Input dimension
  L = mean.shape[0]

  # Sigma
  lmbd = alpha ** 2 * (L + kappa) - kappa
  sigma = np.sqrt(L * lmbd)

  # Decompose covariance matric cov = L @ L.T
  P = np.linalg.cholesky(cov)

  # Generate sigma-points
  sigma_points = np.zeros((2 * L + 1, L))
  sigma_points[0, :] = mean
  for i in range(0, L):
    sigma_points[i + 1, :] = mean + sigma * P[:, i]
    sigma_points[L + i + 1, :] = mean - sigma * P[:, i]

  # Apply transformation to the sigma points
  transformed_sigma_points = transform_functor(sigma_points)

  # Compute transformation mean
  mean_tr = transformed_sigma_points.T @ mean_weights

  # Compute transformation covariance
  centered_transformed_sigma_points = transformed_sigma_points - mean_tr
  cov_tr = centered_transformed_sigma_points.T @ np.diag(cov_weights) @ centered_transformed_sigma_points

  return mean_tr, cov_tr

In [158]:
def cartesian_to_polar(x):
    y = np.zeros(x.shape)
    y[:, 0] = np.hypot(x[:, 0], x[:, 1])
    y[:, 1] = np.arctan2(x[:, 1], x[:, 0])
    return y

# Input mean and covariancer
mean = np.array([12.3, 7.6])
cov = np.array([[1.44, 0.0], [0.0, 2.89]])
print("input mean =", mean)
print("input covariance =\n", cov)

# Unscented transform parameters
alpha = 1e-2
kappa = -1
beta = 0
input_size = mean.shape[0]

# Unscented transform weights
wm, wc = van_der_merwe_weights(input_size, alpha, kappa, beta)

# Apply unscented transform
tr_mean, tr_cov = unscented_transform(cartesian_to_polar, mean, cov, wm, wc, alpha, kappa)
print("transformation mean =", tr_mean)
print("transformation covariance =\n", tr_cov)



input mean = [12.3  7.6]
input covariance =
 [[1.44 0.  ]
 [0.   2.89]]
transformation mean = [14.51616072  0.55146333]
transformation covariance =
 [[1.22125441 0.02861947]
 [0.02861947 0.0080347 ]]
